In [1]:
!pip install faker
!pip install fastapi uvicorn pyngrok
!pip install dash

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 30.2 MB/s eta 0:00:00


In [2]:
import pandas as pd
from faker import Faker
import random
import numpy as np
import plotly.express as px
import uvicorn
import requests

# Generating synthetic data

In [3]:
# Re-initialize Faker
faker = Faker()

def generate_data(num_records):
    data = []
    locations = ['Location A', 'Location B', 'Location C']  # Only 3 different types of locations
    batches = [f"Batch-{str(i).zfill(2)}" for i in range(1, 11)]  # 10 different types of batches as strings

    for _ in range(num_records):
        location = faker.random_element(elements=locations)
        shift = faker.random_element(elements=('Morning', 'Afternoon', 'Night'))
        line = faker.random_element(elements=('Line 1', 'Line 2', 'Line 3'))
        item = faker.random_element(elements=('Item A', 'Item B', 'Item C'))
        batch = faker.random_element(elements=batches)
        planned_production_time = faker.random_int(min=1, max=8)  # in hours

        # Adjust values to ensure OEE falls within the desired range
        # Setting run time close to planned production time for high availability
        run_time = planned_production_time - faker.random_int(min=0, max=1)  # Almost full availability

        # Adjusting production to ensure performance is high but realistic
        production_capacity = faker.random_int(min=50, max=500)  # units
        total_produced = int(production_capacity * random.uniform(0.85, 0.95))  # High performance

        # Ensuring quality is high
        good_count = int(total_produced * random.uniform(0.92, 0.98))  # High quality

        data.append([location, shift, line, item, batch, planned_production_time, run_time, production_capacity, total_produced, good_count])

    return pd.DataFrame(data, columns=['Location', 'Shift', 'Line', 'Item', 'Batch', 'Planned Production Time', 'Run Time', 'Production Capacity', 'Total Produced', 'Good Count'])

# Generating adjusted synthetic data for 1000 records
df = generate_data(1000)

#Pushing the synthetic data to SQLite

In [4]:
import sqlite3

# Connect to SQLite database (it will be created if it doesn't exist)
conn = sqlite3.connect('synthetic-data.db')

# Convert DataFrame to SQL table
df.to_sql('data_table', conn, if_exists='replace', index=False)

# Close the connection
conn.close()

In [5]:
#Query the data in SQLite
# Connect to the SQLite database
conn = sqlite3.connect('synthetic-data.db')

# Query to fetch data from the table
query = "SELECT * FROM data_table"

# Load the query results into a pandas DataFrame
df_queried = pd.read_sql_query(query, conn)

# Display the first few rows of the DataFrame to inspect
print(df_queried.head())

# Close the database connection
conn.close()

     Location      Shift    Line    Item     Batch  Planned Production Time  \
0  Location C    Morning  Line 1  Item C  Batch-08                        4   
1  Location C    Morning  Line 1  Item A  Batch-02                        8   
2  Location C  Afternoon  Line 2  Item B  Batch-09                        2   
3  Location A    Morning  Line 2  Item B  Batch-10                        8   
4  Location C  Afternoon  Line 3  Item A  Batch-08                        8   

   Run Time  Production Capacity  Total Produced  Good Count  
0         4                  322             291         269  
1         7                  477             413         399  
2         1                  157             133         129  
3         7                   97              84          79  
4         8                  426             366         338  


# Building the API with FastAPI and serving with Uvicorn and Ngrok

In [6]:
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
app = FastAPI()

# Define a Pydantic model for the data structure
class Record(BaseModel):
    Location: str
    Shift: str
    Line: str
    Item: str
    Batch: str
    Planned_Production_Time: int
    Run_Time: int
    Production_Capacity: int
    Total_Produced: int
    Good_Count: int

@app.get("/records/")
async def read_records():
    # Connect to SQLite database
    conn = sqlite3.connect('synthetic-data.db')
    # Read the table into a DataFrame
    df_api = pd.read_sql_query("SELECT * FROM data_table", conn)
    conn.close()
    # Convert DataFrame to list of dictionaries (which FastAPI automatically converts to JSON)
    return df_api.to_dict(orient="records")

In [7]:
from threading import Thread

def run_api():
    uvicorn.run(app, host="0.0.0.0", port=8000, log_level="info")

# Start the FastAPI app in a separate thread
thread = Thread(target=run_api)
thread.start()

In [9]:
from pyngrok import ngrok

# Setup ngrok with your auth token
ngrok.set_auth_token("2cZTEP7whJqVJiMaWdQBmG6KG5F_43kovvsKmSgUKGUsQUXtJ")

# Get the list of active tunnels
active_tunnels = ngrok.get_tunnels()

# Disconnect existing tunnels if any
for tunnel in active_tunnels:
    public_url = tunnel.public_url
    ngrok.disconnect(public_url)
    print(f"Disconnected existing tunnel: {public_url}")

# After disconnecting existing tunnels, start a new ngrok tunnel to the uvicorn server
ngrok_tunnel = ngrok.connect(8000, bind_tls=True)
print('Public URL:', ngrok_tunnel.public_url)

Public URL: https://2271-35-194-33-136.ngrok-free.app


In [10]:
# Make sure to replace 'public_url' with the actual ngrok public URL from your setup
public_url = ngrok_tunnel.public_url  # This should be the ngrok URL you obtained dynamically

# Forming the URL to the endpoint
endpoint_url = f"{public_url}/records/"

# Sending a GET request to the endpoint
response = requests.get(endpoint_url)

# Checking if the request was successful
if response.status_code == 200:
    print("Success! Endpoint responded with data.")
    # Optionally, convert the response to a DataFrame to inspect
    df_api = pd.DataFrame(response.json())
    print(df_api.head())  # Display the first few rows of the DataFrame
else:
    print(f"Failed to retrieve data. Status code: {response.status_code}")


INFO:     35.194.33.136:0 - "GET /records/ HTTP/1.1" 200 OK
Success! Endpoint responded with data.
     Location      Shift    Line    Item     Batch  Planned Production Time  \
0  Location C    Morning  Line 1  Item C  Batch-08                        4   
1  Location C    Morning  Line 1  Item A  Batch-02                        8   
2  Location C  Afternoon  Line 2  Item B  Batch-09                        2   
3  Location A    Morning  Line 2  Item B  Batch-10                        8   
4  Location C  Afternoon  Line 3  Item A  Batch-08                        8   

   Run Time  Production Capacity  Total Produced  Good Count  
0         4                  322             291         269  
1         7                  477             413         399  
2         1                  157             133         129  
3         7                   97              84          79  
4         8                  426             366         338  


In [11]:
import requests

# Automatically assign the ngrok public URL to the variable
public_url = ngrok_tunnel.public_url

response = requests.get(f"{public_url}/records/")
df_api = pd.DataFrame(response.json())

# Now df_api contains the data fetched from your FastAPI app

INFO:     35.194.33.136:0 - "GET /records/ HTTP/1.1" 200 OK


# Plotting the charts for OEE Dashboard

In [12]:
# Calculate OEE components in percentage
df_api['Availability'] = (df_api['Run Time'] / df_api['Planned Production Time']) * 100
df_api['Performance'] = (df_api['Total Produced'] / df_api['Production Capacity']) * 100
df_api['Quality'] = (df_api['Good Count'] / df_api['Total Produced']) * 100

# Calculate OEE in percentage, ensuring it falls within the 70% to 88% range
df_api['OEE'] = df_api['Availability'] * df_api['Performance'] * df_api['Quality'] / 10000  # Adjusted for percentage calculation

## Functions for adding target lines and moving average

In [13]:
def add_target_line(fig, target=85, orientation='horizontal'):
    """
    Adds a target line to a plotly figure.

    Parameters:
    - fig: The plotly figure to which the target line will be added.
    - target: The target value for the OEE (default is 85).
    - orientation: The orientation of the chart ('horizontal' for bar charts with horizontal bars,
                   'vertical' for bar charts with vertical bars or scatter plots).
    """
    line_color = "red"
    line_dash = "dash"
    annotation_text = f"Target OEE {target}%"

    if orientation == 'horizontal':
        fig.add_vline(x=target, line_dash=line_dash, line_color=line_color,
                      annotation_text=annotation_text, annotation_position="top right")
    else:
        fig.add_hline(y=target, line_dash=line_dash, line_color=line_color,
                      annotation_text=annotation_text, annotation_position="top right")

In [14]:
import plotly.graph_objects as go

def add_moving_average_trendline(fig, df, x, y, orientation='vertical', window_size=3, ma_color='green'):
    """
    Adds a moving average trendline to a Plotly figure for both vertical and horizontal bar charts.

    Parameters:
    - fig: The Plotly figure to which the moving average trendline will be added.
    - df: DataFrame containing the data.
    - x: Column name for the primary axis (x-axis for vertical, y-axis for horizontal).
    - y: Column name for the values to calculate the moving average.
    - orientation: 'vertical' for vertical bar charts, 'horizontal' for horizontal bar charts.
    - window_size: Size of the moving window for the moving average calculation.
    - ma_color: Color of the moving average trendline.
    """
    # Ensure DataFrame is sorted by the specified axis column to correctly apply the moving average
    df_sorted = df.sort_values(by=x).copy()

    # Calculate the moving average
    df_sorted['Moving_Average'] = df_sorted[y].rolling(window=window_size, min_periods=1).mean()

    if orientation == 'vertical':
        fig.add_trace(go.Scatter(x=df_sorted[x], y=df_sorted['Moving_Average'], mode='lines',
                                 name='Moving Average', line=dict(color=ma_color, width=2.5)))
    elif orientation == 'horizontal':
        fig.add_trace(go.Scatter(x=df_sorted['Moving_Average'], y=df_sorted[x], mode='lines',
                                 name='Moving Average', line=dict(color=ma_color, width=2.5)))

## Charts

In [15]:
# Defining a beautiful color scheme for discrete values
color_scheme = px.colors.qualitative.Pastel

In [16]:
fig_location = px.bar(df_api.groupby('Location')['OEE'].mean().reset_index(), x='Location', y='OEE',
                      title="OEE by Location",
                      color='Location',  # Apply color based on unique values of location
                      labels={"OEE": "Overall Equipment Effectiveness (%)"},
                      color_discrete_sequence=color_scheme)
fig_location.update_layout(xaxis_title="Location", yaxis_title="OEE (%)")
add_target_line(fig_location, target=85, orientation='vertical')  # Add target line
df_api_location = df_api.groupby('Location')['OEE'].mean().reset_index()
add_moving_average_trendline(fig_location, df_api_location, 'Location', 'OEE', window_size=3, ma_color='green')
fig_location.show()

In [17]:
fig_shift = px.pie(df_api.groupby('Shift')['OEE'].mean().reset_index(), names='Shift', values='OEE',
                   title="OEE by Shift",
                   color_discrete_sequence=color_scheme)  # Apply beautiful color scheme
fig_shift.update_traces(textposition='inside', textinfo='percent+label')
fig_shift.show()

In [18]:
# OEE by Item
fig_item = px.bar(df_api.groupby('Item')['OEE'].mean().reset_index(), y='Item', x='OEE',
                  title="OEE by Item",
                  color='Item',  # Apply color based on unique values of item
                  orientation='h',
                  labels={"OEE": "Overall Equipment Effectiveness (%)"},
                  color_discrete_sequence=color_scheme)
fig_item.update_layout(yaxis_title="Item", xaxis_title="OEE (%)")
add_target_line(fig_item, target=85, orientation='horizontal')  # Add target line
df_api_item = df_api.groupby('Item')['OEE'].mean().reset_index()
add_moving_average_trendline(fig_item, df_api_item, 'Item', 'OEE', orientation='horizontal', window_size=3, ma_color='green')
fig_item.show()

In [19]:
# Calculate average OEE by batch
batch_avg_oee = df_api.groupby('Batch')['OEE'].mean().reset_index()

fig_batch = px.bar(batch_avg_oee, x='Batch', y='OEE',
                   title="OEE by Batch",
                   color='Batch',  # Apply color based on different batch
                   labels={"OEE": "Overall Equipment Effectiveness (%)"},
                   color_discrete_sequence=color_scheme)
fig_batch.update_layout(xaxis_title="Batch", yaxis_title="OEE (%)", xaxis={'categoryorder':'category ascending'})
add_target_line(fig_batch, target=85, orientation='vertical')  # Add target line
add_moving_average_trendline(fig_batch, batch_avg_oee, 'Batch', 'OEE', orientation='vertical', window_size=3, ma_color='green')
fig_batch.show()

# Building a Dash App

In [22]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import requests

# Initialize the Dash app
app = dash.Dash(__name__)

# Define the layout of the app
app.layout = html.Div([
    html.H1("OEE Dashboard"),
    dcc.Graph(id="graph-location"),
    dcc.Graph(id="graph-shift"),
    dcc.Graph(id="graph-item"),
    dcc.Graph(id="graph-batch"),
    dcc.Interval(
        id="update-interval",
        interval=30*1000,  # in milliseconds (30 seconds)
        n_intervals=0
    )
])

# Replace with your actual ngrok public URL
API_URL = "https://2271-35-194-33-136.ngrok-free.app/records/"

# Defining a beautiful color scheme for discrete values
color_scheme = px.colors.qualitative.Pastel

def fetch_data():
    response = requests.get(API_URL)
    if response.status_code == 200:
        data = response.json()
        df = pd.DataFrame(data)
        # Calculate OEE components in percentage and OEE
        df['Availability'] = (df['Run Time'] / df['Planned Production Time']) * 100
        df['Performance'] = (df['Total Produced'] / df['Production Capacity']) * 100
        df['Quality'] = (df['Good Count'] / df['Total Produced']) * 100
        df['OEE'] = df['Availability'] * df['Performance'] * df['Quality'] / 10000
        return df
    else:
        raise Exception("API request failed")

# Callback for updating the graph-location
@app.callback(Output('graph-location', 'figure'),
              Input('update-interval', 'n_intervals'))
def update_graph_location(n):
    df = fetch_data()
    # Group by Location and calculate mean OEE
    df_grouped = df.groupby('Location', as_index=False)['OEE'].mean()
    # Defining a beautiful color scheme for discrete values
    color_scheme = px.colors.qualitative.Pastel
    # Create the bar chart, using 'Location' as the color category
    fig = px.bar(df_grouped, x='Location', y='OEE',
                 title="OEE by Location",
                 labels={"OEE": "Overall Equipment Effectiveness (%)"},
                 color='Location',  # This specifies that the color should vary by location
                 color_discrete_sequence=color_scheme)
    fig.update_layout(xaxis_title="Location", yaxis_title="OEE (%)")
    return fig

# Callback for updating the graph-shift
@app.callback(Output('graph-shift', 'figure'),
              Input('update-interval', 'n_intervals'))
def update_graph_shift(n):
    df = fetch_data()
    fig = px.pie(df.groupby('Shift')['OEE'].mean().reset_index(), names='Shift', values='OEE',
                 title="OEE by Shift", color='Shift', color_discrete_sequence=color_scheme)
    fig.update_traces(textposition='inside', textinfo='percent+label')
    return fig

# Callback for updating the graph-item
@app.callback(Output('graph-item', 'figure'),
              Input('update-interval', 'n_intervals'))
def update_graph_item(n):
    df = fetch_data()
    fig = px.bar(df.groupby('Item')['OEE'].mean().reset_index(), y='Item', x='OEE',
                 title="OEE by Item", orientation='h', labels={"OEE": "Overall Equipment Effectiveness (%)"}, color='Item',
                 color_discrete_sequence=color_scheme)
    fig.update_layout(yaxis_title="Item", xaxis_title="OEE (%)")
    return fig

# Callback for updating the graph-batch
@app.callback(Output('graph-batch', 'figure'),
              Input('update-interval', 'n_intervals'))
def update_graph_batch(n):
    df = fetch_data()
    fig = px.bar(df.groupby('Batch')['OEE'].mean().reset_index(), x='Batch', y='OEE',
                 title="OEE by Batch", labels={"OEE": "Overall Equipment Effectiveness (%)"}, color='Batch',
                 color_discrete_sequence=color_scheme)
    fig.update_layout(xaxis_title="Batch", yaxis_title="OEE (%)", xaxis={'categoryorder':'category ascending'})
    return fig

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)

<IPython.core.display.Javascript object>